# <center>Implementing LDA in Python</center>

## Key Concepts in this Notebook

## Introduction

## Importing the Required Libraries

In [5]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Preparing the Data

In [6]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [8]:
stopwords = stopwords.words("english")

In [9]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
data = load_data("data/ushmm_dn.json")["texts"]

print (data[0][0:90])

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928. 


In [13]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load(name = "en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

name bear small town call bear very hard work child father mother small mill flour buckwhe


In [14]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [15]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [16]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow


In [17]:
# id2word = corpora.Dictionary(all_texts)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

# print (corpus[0][0:20])

# word = id2word[[0][:1][0]]
# print (word)

In [18]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


In [19]:
test_doc = corpus[-1]

vector = lda_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.022351041), (1, 0.33406878), (2, 0.04829154), (3, 0.033661727), (5, 0.02003186), (6, 0.023813836), (7, 0.49504378), (9, 0.013045194)]
[(7, 0.49504378), (1, 0.33406878), (2, 0.04829154), (3, 0.033661727), (6, 0.023813836), (0, 0.022351041), (5, 0.02003186), (9, 0.013045194)]


In [23]:
from pathlib import Path

In [24]:
main_path = Path.cwd()
main_path

WindowsPath('C:/Users/Kartik Yanigar/Desktop/Class/Projects/Placement_Resume_Projects/topic_modelling')

In [26]:
lda_model.save(main_path.joinpath("models/test_model.model").as_posix())

In [27]:
new_model = gensim.models.ldamodel.LdaModel.load(main_path.joinpath("models/test_model.model").as_posix())

In [28]:
test_doc = corpus[-1]

vector = new_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.022351786), (1, 0.3340695), (2, 0.04828691), (3, 0.033661947), (5, 0.020031933), (6, 0.023813648), (7, 0.4950439), (9, 0.013046633)]
[(7, 0.4950439), (1, 0.3340695), (2, 0.04828691), (3, 0.033661947), (6, 0.023813648), (0, 0.022351786), (5, 0.020031933), (9, 0.013046633)]


## Vizualizing the Data

In [31]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30, n_jobs = 1)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.002486  0.121470       1        1  22.036530
1      0.112606  0.316816       2        1  18.608958
7     -0.330085 -0.115820       3        1  15.260569
3     -0.192040  0.285610       4        1  10.697965
9     -0.120287 -0.265022       5        1   8.803878
0      0.054897 -0.104349       6        1   6.370592
8      0.107005 -0.315641       7        1   5.738729
5     -0.261657  0.088890       8        1   5.269022
6      0.300106 -0.147501       9        1   4.970453
4      0.331940  0.135547      10        1   2.243303, topic_info=          Term         Freq        Total Category  logprob  loglift
337     ghetto  2306.000000  2306.000000  Default  30.0000  30.0000
187     course  1371.000000  1371.000000  Default  29.0000  29.0000
663   remember  1085.000000  1085.000000  Default  28.0000  28.0000
852        use  1652.000000  1652.000000  Default  27.0000  27.0000
569       over   643.000000   643.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
912     action    25.368419   120.616163  Topic10  -5.1355   2.2381
957     arrive    28.792160   257.321817  Topic10  -5.0089   1.6070
350      great    26.183468   258.135610  Topic10  -5.1039   1.5089
547         of    25.128249   326.161044  Topic10  -5.1450   1.2338
1479    lawyer    22.901819   141.946009  Topic10  -5.2378   1.9730

[700 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
0          2  0.330324            able
0          3  0.004434            able
0          5  0.467775            able
0          6  0.039905            able
0          7  0.157403            able
...      ...       ...             ...
10173      2  0.970564  youth_movement
2551       2  0.016739         zionist
2551       3  0.050216         zionist
2551       6  0.100431         zionist
2551       9  0.820190         zionist

[1538 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 8, 4, 10, 1, 9, 6, 7, 5])